## Load documents with IDs

In [3]:
import json

with open('/home/tinchung/Documents/GitHub/End-to-end-law-searching-with-RAG-Mamba/documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [4]:
documents[10]

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 11',
 'title': 'Ngôn ngữ, chữ viết dùng trong cơ sở giáo dục',
 'content': '1. Tiếng Việt là ngôn ngữ chính thức dùng trong cơ sở giáo dục. Căn cứ vào mục tiêu giáo dục và yêu cầu cụ thể về nội dung giáo dục, Chính phủ quy định việc dạy và học bằng tiếng nước ngoài trong cơ sở giáo dục. 2. Nhà nước khuyến khích, tạo điều kiện để người dân tộc thiểu số được học tiếng nói, chữ viết của dân tộc mình theo quy định của Chính phủ; người khuyết tật nghe, nói được học bằng ngôn ngữ ký hiệu, người khuyết tật nhìn được học bằng chữ nổi Braille theo quy định của Luật Người khuyết tật. 3. Ngoại ngữ quy định trong chương trình giáo dục là ngôn ngữ được sử dụng phổ biến trong giao dịch quốc tế. Việc tổ chức dạy ngoại ngữ trong cơ sở giáo dục phải bảo đảm để người học được học liên tục, hiệu quả. ',
 'id': '1095c99a'}

## Load ground truth

In [5]:
import pandas as pd

df_ground_truth = pd.read_csv('./ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')

In [6]:
ground_truth[10]

{'questions': 'Nền giáo dục Việt Nam có đặc điểm gì nổi bật?',
 'law_title': 'Luật Giáo Dục',
 'document': 'e0e71ffa'}

In [7]:
doc_idx = {d['id']: d for d in documents}
doc_idx['e0e71ffa']

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 3',
 'title': 'Tính chất, nguyên lý giáo dục',
 'content': '1. Nền giáo dục Việt Nam là nền giáo dục xã hội chủ nghĩa có tính nhân dân, dân tộc, khoa học, hiện đại, lấy chủ nghĩa Mác - Lê nin và tư tưởng Hồ Chí Minh làm nền tảng. 2. Hoạt động giáo dục được thực hiện theo nguyên lý học đi đôi với hành, lý luận gắn liền với thực tiễn, giáo dục nhà trường kết hợp với giáo dục gia đình và giáo dục xã hội. ',
 'id': 'e0e71ffa'}

## Index data

In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("truro7/vn-law-embedding", truncate_dim = 768) #truncate_dim = 768 

Invalid model-index. Not loading eval results into CardData.
Invalid model-index. Not loading eval results into CardData.


In [9]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

# es_client.info()
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "law_title": {"type": "keyword"},                
            "law_number": {"type": "keyword"},               
            "chapter_title": {"type": "keyword"},            
            "article_number": {"type": "keyword"},            
            "title": {"type": "text"},                        
            "content": {"type": "text"},   
            "id": {"type": "text"},   
            "title_content_vector": {                                 
                "type": "dense_vector",
                "dims": 768,                                 
                "index": True,
                "similarity": "cosine"
            }                
        }
    }
}

index_name = "legal_documents_ids"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'legal_documents_ids'})

In [10]:
documents[10]

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 11',
 'title': 'Ngôn ngữ, chữ viết dùng trong cơ sở giáo dục',
 'content': '1. Tiếng Việt là ngôn ngữ chính thức dùng trong cơ sở giáo dục. Căn cứ vào mục tiêu giáo dục và yêu cầu cụ thể về nội dung giáo dục, Chính phủ quy định việc dạy và học bằng tiếng nước ngoài trong cơ sở giáo dục. 2. Nhà nước khuyến khích, tạo điều kiện để người dân tộc thiểu số được học tiếng nói, chữ viết của dân tộc mình theo quy định của Chính phủ; người khuyết tật nghe, nói được học bằng ngôn ngữ ký hiệu, người khuyết tật nhìn được học bằng chữ nổi Braille theo quy định của Luật Người khuyết tật. 3. Ngoại ngữ quy định trong chương trình giáo dục là ngôn ngữ được sử dụng phổ biến trong giao dịch quốc tế. Việc tổ chức dạy ngoại ngữ trong cơ sở giáo dục phải bảo đảm để người học được học liên tục, hiệu quả. ',
 'id': '1095c99a'}

In [11]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    title = doc['title']
    content = doc['content']
    doc['title_content_vector'] = model.encode(title + ' ' + content)

    es_client.index(index=index_name, document=doc)

  0%|          | 0/314 [00:00<?, ?it/s]

## Retrieval

In [12]:
def elastic_search_knn(field, vector, law_title):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "law_title": law_title
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["law_title", "law_number", "chapter_title", "article_number", "title", "content","id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

def title_content_vector_knn(q):
    questions = q['questions']
    law_title = q['law_title']

    v_q = model.encode(questions)

    return elastic_search_knn('title_content_vector', v_q, law_title)


In [14]:
title_content_vector_knn(dict(
    questions='Khi nào thì người điều khiển được phép lùi xe?',
    law_title='Luật Giao Thông Đường Bộ'
))

[{'article_number': 'Điều 16',
  'law_number': 'Luật số: 23/2008/QH12',
  'chapter_title': 'CHƯƠNG II',
  'id': '7f7a2ff6',
  'title': 'Lùi xe',
  'law_title': 'Luật Giao Thông Đường Bộ',
  'content': '1. Khi lùi xe, người điều khiển phải quan sát phía sau, có tín hiệu cần thiết và chỉ khi nào thấy không nguy hiểm mới được lùi. 2. Không được lùi xe ở khu vực cấm dừng, trên phần đường dành cho người đi bộ qua đường, nơi đường bộ giao nhau, đường bộ giao nhau cùng mức với đường sắt, nơi tầm nhìn bị che khuất, trong hầm đường bộ, đường cao tốc. '},
 {'article_number': 'Điều 15',
  'law_number': 'Luật số: 23/2008/QH12',
  'chapter_title': 'CHƯƠNG II',
  'id': '4fde6f0d',
  'title': 'Chuyển hướng xe',
  'law_title': 'Luật Giao Thông Đường Bộ',
  'content': '1. Khi muốn chuyển hướng, người điều khiển phương tiện phải giảm tốc độ và có tín hiệu báo hướng rẽ. 2. Trong khi chuyển hướng, người lái xe, người điều khiển xe máy chuyên dùng phải nhường quyền đi trước cho người đi bộ, người đi xe đạp

## The RAG flow

In [15]:
def build_prompt(query, search_results):
    prompt_template = """
You're a law searcher assistant. Answer the QUESTION based on the CONTEXT from the law database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context += f"law_title: {doc['law_title']}\nlaw_number: {doc['law_number']}\narticle_number: {doc['article_number']}\ntitle: {doc['title']}\ncontent: {doc['content']}\n\n"

    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt



In [21]:
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

# Set the maximum token length to 4096
config = MambaConfig(max_length=4096)
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
model = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf")

In [22]:
def predict_mamba_130m(prompt):
    prefix = "You are a helpful assistant. Respond to the following query: "
    full_prompt = prefix + prompt
    input_ids = tokenizer(full_prompt, return_tensors="pt")["input_ids"]

    # Generate output with sampling
    out = model.generate(
        input_ids,
        max_new_tokens=50,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

    # Decode the output
    decoded_output = tokenizer.batch_decode(out)[0]

    # Remove the full prompt (including prefix)
    answer = decoded_output[len(full_prompt):]

    # Remove repeating phrases (if present)
    phrases = answer.split(".")  # Split into phrases based on periods
    unique_phrases = []
    for phrase in phrases:
        phrase = phrase.strip()
        if phrase not in unique_phrases and phrase:  # Check for emptiness too
            unique_phrases.append(phrase)
    answer = ". ".join(unique_phrases)

    return answer
prompt = "Hey, how are you doing?"
generated_answer = predict_mamba_130m(prompt)
print(generated_answer)

How are you?

4. What is your favorite activity?

5. What do you like to do most?

6. What is the most fun you have had in your working life?

7. How many hours


In [ ]:
# previously: rag(query: str) -> str
def rag(query: dict, model='gpt-4o') -> str:
    search_results = title_content_vector_knn(query)
    prompt = build_prompt(query['question'], search_results)
    answer = llm(prompt, model=model)
    return answer

In [ ]:
ground_truth[10]

In [ ]:
rag(ground_truth[10])

In [ ]:
doc_idx['5170565b']['text']

## Cosine similarity metric

In [ ]:
answer_orig = 'Yes, sessions are recorded if you miss one. Everything is recorded, allowing you to catch up on any missed content. Additionally, you can ask questions in advance for office hours and have them addressed during the live stream. You can also ask questions in Slack.'
answer_llm = 'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.'

v_llm = model.encode(answer_llm)
v_orig = model.encode(answer_orig)

v_llm.dot(v_orig)

In [ ]:
ground_truth[0]

In [ ]:
len(ground_truth)

In [ ]:
rec

In [ ]:
answers = {}

In [ ]:
for i, rec in enumerate(tqdm(ground_truth)):
    if i in answers:
        continue

    answer_llm = rag(rec)
    doc_id = rec['document']
    original_doc = doc_idx[doc_id]
    answer_orig = original_doc['text']

    answers[i] = {
        'answer_llm': answer_llm,
        'answer_orig': answer_orig,
        'document': doc_id,
        'question': rec['question'],
        'course': rec['course'],
    }

In [ ]:
results_gpt4o = [None] * len(ground_truth)

for i, val in answers.items():
    results_gpt4o[i] = val.copy()
    results_gpt4o[i].update(ground_truth[i])

In [ ]:
import pandas as pd

In [ ]:
df_gpt4o = pd.DataFrame(results_gpt4o)

In [ ]:
!mkdir data

In [ ]:
df_gpt4o.to_csv('data/results-gpt4o.csv', index=False)

## Evaluating GPT 3.5

In [ ]:
rag(ground_truth[10], model='gpt-3.5-turbo')

In [ ]:
from tqdm.auto import tqdm

from concurrent.futures import ThreadPoolExecutor

pool = ThreadPoolExecutor(max_workers=6)

def map_progress(pool, seq, f):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results

In [ ]:
def process_record(rec):
    model = 'gpt-3.5-turbo'
    answer_llm = rag(rec, model=model)
    
    doc_id = rec['document']
    original_doc = doc_idx[doc_id]
    answer_orig = original_doc['text']

    return {
        'answer_llm': answer_llm,
        'answer_orig': answer_orig,
        'document': doc_id,
        'question': rec['question'],
        'course': rec['course'],
    }

In [ ]:
process_record(ground_truth[10])

In [ ]:
results_gpt35 = map_progress(pool, ground_truth, process_record)

In [ ]:
df_gpt35 = pd.DataFrame(results_gpt35)
df_gpt35.to_csv('data/results-gpt35.csv', index=False)

In [ ]:
!head data/results-gpt35.csv

## Cosine similarity

A->Q->A' cosine similarity

A -> Q -> A'

cosine(A, A')

### gpt-4o

In [ ]:
results_gpt4o = df_gpt4o.to_dict(orient='records')

In [ ]:
record = results_gpt4o[0]

In [ ]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = model.encode(answer_llm)
    v_orig = model.encode(answer_orig)
    
    return v_llm.dot(v_orig)

In [ ]:
similarity = []

for record in tqdm(results_gpt4o):
    sim = compute_similarity(record)
    similarity.append(sim)

In [ ]:
df_gpt4o['cosine'] = similarity
df_gpt4o['cosine'].describe()

In [ ]:
import seaborn as sns

### gpt-3.5-turbo

In [ ]:
results_gpt35 = df_gpt35.to_dict(orient='records')

similarity_35 = []

for record in tqdm(results_gpt35):
    sim = compute_similarity(record)
    similarity_35.append(sim)

In [ ]:
df_gpt35['cosine'] = similarity_35
df_gpt35['cosine'].describe()

In [ ]:
import matplotlib.pyplot as plt

### gpt-4o-mini

In [ ]:
def process_record_4o_mini(rec):
    model = 'gpt-4o-mini'
    answer_llm = rag(rec, model=model)
    
    doc_id = rec['document']
    original_doc = doc_idx[doc_id]
    answer_orig = original_doc['text']

    return {
        'answer_llm': answer_llm,
        'answer_orig': answer_orig,
        'document': doc_id,
        'question': rec['question'],
        'course': rec['course'],
    }

In [ ]:
process_record_4o_mini(ground_truth[10])

In [ ]:
results_gpt4omini = []

In [ ]:
for record in tqdm(ground_truth):
    result = process_record_4o_mini(record)
    results_gpt4omini.append(result)

In [ ]:
df_gpt4o_mini = pd.DataFrame(results_gpt4omini)
df_gpt4o_mini.to_csv('data/results-gpt4o-mini.csv', index=False)

In [ ]:
similarity_4o_mini = []

for record in tqdm(results_gpt4omini):
    sim = compute_similarity(record)
    similarity_4o_mini.append(sim)

In [ ]:
df_gpt4o_mini['cosine'] = similarity_4o_mini
df_gpt4o_mini['cosine'].describe()

gpt4o 

```
count    1830.000000
mean        0.679129
std         0.217995
min        -0.153426
25%         0.591460
50%         0.734788
75%         0.835390
max         0.995339
Name: cosine, dtype: float64
```

In [ ]:
# sns.distplot(df_gpt35['cosine'], label='3.5')

sns.distplot(df_gpt4o['cosine'], label='4o')
sns.distplot(df_gpt4o_mini['cosine'], label='4o-mini')

plt.title("RAG LLM performance")
plt.xlabel("A->Q->A' Cosine Similarity")
plt.legend()

## LLM-as-a-Judge

In [ ]:
prompt1_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer compared to the original answer provided.
Based on the relevance and similarity of the generated answer to the original answer, you will classify
it as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Original Answer: {answer_orig}
Generated Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the original
answer and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

prompt2_template = """
You are an expert evaluator for a Retrieval-Augmented Generation (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [ ]:
df_sample = df_gpt4o_mini.sample(n=150, random_state=1)

In [ ]:
samples = df_sample.to_dict(orient='records')

In [ ]:
record = samples[0]
record

In [ ]:
prompt = prompt1_template.format(**record)
print(prompt)

In [ ]:
answer = llm(prompt, model='gpt-4o-mini')

In [ ]:
import json

In [ ]:
evaluations = []

for record in tqdm(samples):
    prompt = prompt1_template.format(**record)
    evaluation = llm(prompt, model='gpt-4o-mini')
    evaluations.append(evaluation)

In [ ]:
json_evaluations = []

for i, str_eval in enumerate(evaluations):
    json_eval = json.loads(str_eval)
    json_evaluations.append(json_eval)

In [ ]:
df_evaluations = pd.DataFrame(json_evaluations)

In [ ]:
df_evaluations.Relevance.value_counts()

In [ ]:
df_evaluations[df_evaluations.Relevance == 'NON_RELEVANT'] #.to_dict(orient='records')

In [ ]:
sample[4]

In [ ]:
prompt = prompt2_template.format(**record)
print(prompt)

In [ ]:
evaluation = llm(prompt, model='gpt-4o-mini')
print(evaluation)

In [ ]:
evaluations_2 = []

for record in tqdm(samples):
    prompt = prompt2_template.format(**record)
    evaluation = llm(prompt, model='gpt-4o-mini')
    evaluations_2.append(evaluation)

In [ ]:
json_evaluations_2 = []

for i, str_eval in enumerate(evaluations_2):
    json_eval = json.loads(str_eval)
    json_evaluations_2.append(json_eval)

In [ ]:
df_evaluations_2 = pd.DataFrame(json_evaluations_2)

In [ ]:
df_evaluations_2[df_evaluations_2.Relevance == 'NON_RELEVANT']

In [ ]:
samples[45]

## Saving all the data

In [ ]:
df_gpt4o.to_csv('data/results-gpt4o-cosine.csv', index=False)
df_gpt35.to_csv('data/results-gpt35-cosine.csv', index=False)
df_gpt4o_mini.to_csv('data/results-gpt4o-mini-cosine.csv', index=False)

In [ ]:
df_evaluations.to_csv('data/evaluations-aqa.csv', index=False)
df_evaluations_2.to_csv('data/evaluations-qa.csv', index=False)